In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import torch


In [2]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from matplotlib.pyplot import imshow
from PIL import Image
import os
import numpy as np
import json
from detectron2.structures import BoxMode
def cv2_imshow(img):
    img = img[:,:,[2,1,0]]
    img = Image.fromarray(img)
    display(img)
    '''plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    #img = transforms.ToPILImage(img)
    #plt.show(img)
    '''


In [3]:
people_jsonlist = []
people_imagelist = []
with open('/root/notebooks/Module_final/pedestrian_generator/Detectron2/people_jsonlist_final_v2', 'r') as j:
    lines = j.readlines()
    for line in lines:
        people_jsonlist.append(line[:-1])
with open('/root/notebooks/Module_final/pedestrian_generator/Detectron2/people_imagelist_final_v2', 'r') as i:
    lines = i.readlines()
    for line in lines:
        people_imagelist.append(line[:-1])

print(len(people_jsonlist), len(people_imagelist))
print(people_imagelist[:2])
print(people_jsonlist[:2])

46024 46024
['/root/notebooks/Module_final/caltech_origin_data_refine/street/045034.jpg', '/root/notebooks/Module_final/caltech_origin_data_refine/street/031156.jpg']
['/root/notebooks/Module_final/caltech_origin_data_refine/street_json/045034.json', '/root/notebooks/Module_final/caltech_origin_data_refine/street_json/031156.json']


In [4]:
train_people_jsonlist = people_jsonlist[:-4024]
train_people_imagelist = people_imagelist[:-4024]
test_people_jsonlist = people_jsonlist[-4024:]
test_people_imagelist = people_imagelist[-4024:]
print(len(train_people_imagelist))
print(len(train_people_jsonlist))

print(len(test_people_imagelist))
print(len(test_people_jsonlist))

print(test_people_jsonlist[0])

42000
42000
4024
4024
/root/notebooks/Module_final/caltech_origin_data_refine/street_json/096474.json


In [5]:
def get_pedestrain_dict(image_list, json_list):
    dataset_dicts = []
    
    for i,path in tqdm(enumerate(image_list)):
        filename = path
        #img = cv2.imread(path)
        # height, width = cv2.imread(filename).shape[:2]
        record = {}
        record['file_name'] = filename
        record['image_id'] = i #path.split('/')[-1][:-5]
        #id is like 000000 or 000001
        #record['file_img'] = img
        record['height']= 480
        record['width']= 640
        
        #for i in data_list[1] to get bbox and category
        objs = []
        
        people = json_list[i]
        with open(people) as p:
            json_context = json.load(p)
            for person in json_context:
                boxes = list(map(float, person['pos']))
                obj = {
                    "bbox": boxes,
                    "bbox_mode": BoxMode.XYWH_ABS,
                    #"segmentation": [poly], To draw a line, along to ballon
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts #list of dicts

In [6]:
from detectron2.data import DatasetCatalog, MetadataCatalog

DatasetCatalog.register("pedestrain_test", lambda i=i: get_pedestrain_dict(test_people_imagelist, test_people_jsonlist))
MetadataCatalog.get("pedestrain_test").set(thing_classes=["person"])

Metadata(name='pedestrain_test', thing_classes=['person'])

In [13]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Base-RCNN-FPN.yaml"))
#cfg.DATASETS.TRAIN = ("pedestrain_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"#model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
#cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER =  156000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.OUTPUT_DIR = '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/'

In [28]:
import glob
model_list = sorted(glob.glob('/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model*'))
print(model_list[10:])
print(len(model_list))

['/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0084999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0089999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0094999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0099999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0104999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0109999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0114999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_0119999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/mo

In [34]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
evaluation_list = []
for model in model_list:
    cfg.MODEL.WEIGHTS = model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
    cfg.DATASETS.TEST = ("pedestrain_test", )
    
    predictor = DefaultPredictor(cfg)
    COCOEvaluator.reset
    evaluator = COCOEvaluator("pedestrain_test", cfg, False, output_dir = cfg.OUTPUT_DIR)

    val_loader = build_detection_test_loader(cfg, "pedestrain_test")
    evaluation_list.append(inference_on_dataset(predictor.model, val_loader, evaluator))

4024it [00:06, 612.08it/s]


[06/27 17:59:38 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 17:59:38 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 17:59:38 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 17:59:39 d2.evaluation.evaluator]: Inference done 11/4024. 0.0390 s / img. ETA=0:02:41
[06/27 17:59:44 d2.evaluation.evaluator]: Inference done 135/4024. 0.0391 s / img. ETA=0:02:36
[06/27 17:59:49 d2.evaluation.evaluator]: Inference done 258/4024. 0.0392 s / img. ETA=0:02:32
[06/27 17:59:54 d2.evaluation.evaluator]: Inference done 368/4024. 0.0407 s / img. ETA=0:02:33
[06/27 17:59:59 d2.evaluation.evaluator]: Inference done 490/4024. 0.0404 s / img. ETA=0:02:27
[06/27 18:00:04 d2.evaluation.evaluator]: Inference done 584/4024. 0.0425 s / img. ETA=0:02:30
[06/27 18:00:09 d2.evaluation.evaluator]: Inference done 617/4024. 0.0479 s / img. ETA=0:02:49
[06/27 18:00:14 d2.evaluation.evaluator]: Inference done 648/4024. 0.0528 s / img. ETA=0:0

4024it [00:23, 174.46it/s]


[06/27 18:10:18 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 18:10:18 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 18:10:18 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 18:10:21 d2.evaluation.evaluator]: Inference done 11/4024. 0.1662 s / img. ETA=0:12:06
[06/27 18:10:26 d2.evaluation.evaluator]: Inference done 45/4024. 0.1450 s / img. ETA=0:10:10
[06/27 18:10:31 d2.evaluation.evaluator]: Inference done 72/4024. 0.1587 s / img. ETA=0:11:00
[06/27 18:10:36 d2.evaluation.evaluator]: Inference done 103/4024. 0.1563 s / img. ETA=0:10:49
[06/27 18:10:41 d2.evaluation.evaluator]: Inference done 135/4024. 0.1557 s / img. ETA=0:10:39
[06/27 18:10:46 d2.evaluation.evaluator]: Inference done 167/4024. 0.1544 s / img. ETA=0:10:28
[06/27 18:10:51 d2.evaluation.evaluator]: Inference done 198/4024. 0.1557 s / img. ETA=0:10:24
[06/27 18:10:57 d2.evaluation.evaluator]: Inference done 226/4024. 0.1584 s / img. ETA=0:10:

4024it [00:23, 171.99it/s]


...

[06/27 19:45:51 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 19:45:51 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 19:45:51 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 19:45:52 d2.evaluation.evaluator]: Inference done 11/4024. 0.0395 s / img. ETA=0:02:43
[06/27 19:45:57 d2.evaluation.evaluator]: Inference done 134/4024. 0.0395 s / img. ETA=0:02:39
[06/27 19:46:02 d2.evaluation.evaluator]: Inference done 257/4024. 0.0395 s / img. ETA=0:02:33
[06/27 19:46:07 d2.evaluation.evaluator]: Inference done 380/4024. 0.0395 s / img. ETA=0:02:28
[06/27 19:46:12 d2.evaluation.evaluator]: Inference done 503/4024. 0.0394 s / img. ETA=0:02:23
[06/27 19:46:17 d2.evaluation.evaluator]: Inference done 626/4024. 0.0394 s / img. ETA=0:02:18
[06/27 19:46:22 d2.evaluation.evaluator]: Inference done 748/4024. 0.0394 s / img. ETA=0:02:13
[06/27 19:46:27 d2.evaluation.evaluator]: Inference done 871/4024. 0.0394 s / img. ETA=0:0

4024it [00:06, 629.72it/s]


[06/27 19:48:49 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 19:48:49 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 19:48:49 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 19:48:50 d2.evaluation.evaluator]: Inference done 11/4024. 0.0400 s / img. ETA=0:02:45
[06/27 19:48:55 d2.evaluation.evaluator]: Inference done 134/4024. 0.0394 s / img. ETA=0:02:38
[06/27 19:49:00 d2.evaluation.evaluator]: Inference done 257/4024. 0.0394 s / img. ETA=0:02:33
[06/27 19:49:05 d2.evaluation.evaluator]: Inference done 380/4024. 0.0394 s / img. ETA=0:02:28
[06/27 19:49:10 d2.evaluation.evaluator]: Inference done 504/4024. 0.0394 s / img. ETA=0:02:23
[06/27 19:49:15 d2.evaluation.evaluator]: Inference done 627/4024. 0.0393 s / img. ETA=0:02:18
[06/27 19:49:20 d2.evaluation.evaluator]: Inference done 750/4024. 0.0394 s / img. ETA=0:02:13
[06/27 19:49:25 d2.evaluation.evaluator]: Inference done 872/4024. 0.0394 s / img. ETA=0:0

4024it [00:06, 584.79it/s]


[06/27 19:51:44 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 19:51:44 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 19:51:44 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 19:51:45 d2.evaluation.evaluator]: Inference done 11/4024. 0.0403 s / img. ETA=0:02:46
[06/27 19:51:50 d2.evaluation.evaluator]: Inference done 134/4024. 0.0396 s / img. ETA=0:02:39
[06/27 19:51:55 d2.evaluation.evaluator]: Inference done 257/4024. 0.0395 s / img. ETA=0:02:34
[06/27 19:52:00 d2.evaluation.evaluator]: Inference done 379/4024. 0.0396 s / img. ETA=0:02:29
[06/27 19:52:05 d2.evaluation.evaluator]: Inference done 502/4024. 0.0395 s / img. ETA=0:02:24
[06/27 19:52:10 d2.evaluation.evaluator]: Inference done 626/4024. 0.0395 s / img. ETA=0:02:18
[06/27 19:52:15 d2.evaluation.evaluator]: Inference done 749/4024. 0.0394 s / img. ETA=0:02:13
[06/27 19:52:20 d2.evaluation.evaluator]: Inference done 872/4024. 0.0394 s / img. ETA=0:0

In [35]:
print(evaluation_list)
print(len(evaluation_list))

[OrderedDict([('bbox', {'AP': 3.0228896654525625, 'AP50': 7.214390289591156, 'AP75': 1.989218219692366, 'APs': 1.741281294415763, 'APm': 4.878469814304964, 'APl': 1.3696369636963694})]), OrderedDict([('bbox', {'AP': 5.324153672946869, 'AP50': 12.840942387847122, 'AP75': 3.5065417244621706, 'APs': 2.702557069579737, 'APm': 8.785605927935105, 'APl': 4.405035051673831})]), OrderedDict([('bbox', {'AP': 5.883921125721576, 'AP50': 13.830074408919371, 'AP75': 3.7722772029036236, 'APs': 3.3751780819810806, 'APm': 9.024068379213155, 'APl': 8.462490110708352})]), OrderedDict([('bbox', {'AP': 7.498841146150956, 'AP50': 16.758429851392645, 'AP75': 5.490231606469758, 'APs': 4.015109565144007, 'APm': 11.420462194837807, 'APl': 12.774737673366518})]), OrderedDict([('bbox', {'AP': 9.511271623711961, 'AP50': 21.937408764431513, 'AP75': 6.228228502128933, 'APs': 5.735081917857228, 'APm': 13.743268393342664, 'APl': 14.466704102965034})]), OrderedDict([('bbox', {'AP': 10.098502961924897, 'AP50': 22.049421

In [38]:
iter_count = 30000
transfer_x_AP, transfer_x_APs, transfer_x_APm, transfer_x_APl = [], [], [], []
transfer_y = []
for evaluation in evaluation_list[:-1]:
#     print(evaluation['bbox']['AP'])
    transfer_x_AP.append(evaluation['bbox']['AP'])
    transfer_x_APs.append(evaluation['bbox']['APs'])
    transfer_x_APm.append(evaluation['bbox']['APm'])
    transfer_x_APl.append(evaluation['bbox']['APl'])
    iter_count+=5000
    transfer_y.append(iter_count)
transfer_x_AP.append(evaluation_list[-1]['bbox']['AP'])
transfer_x_APs.append(evaluation_list[-1]['bbox']['APs'])
transfer_x_APm.append(evaluation_list[-1]['bbox']['APm'])
transfer_x_APl.append(evaluation_list[-1]['bbox']['APl'])
transfer_y.append(126000)
print(transfer_x_AP)
print(transfer_x_APs)
print(transfer_x_APm)
print(transfer_x_APl)
print(transfer_y)


[3.0228896654525625, 5.324153672946869, 5.883921125721576, 7.498841146150956, 9.511271623711961, 10.098502961924897, 10.651938517694015, 10.512437147892891, 13.551946929841902, 13.560070159877636, 15.566629971613805, 15.774787192580012, 18.103501028521524, 18.507908836041448, 18.606010917413727, 18.889177542853027, 18.889781995680586, 18.929545461319357, 18.91732702457656, 18.9339833705924]
[1.741281294415763, 2.702557069579737, 3.3751780819810806, 4.015109565144007, 5.735081917857228, 6.087071336304615, 6.3102875457135985, 7.213730765001965, 9.450606864894578, 8.99690356725498, 10.374319470394493, 11.271325528947095, 13.143792203245658, 13.63008906329275, 13.765369763922813, 13.819029101995767, 14.141578261443518, 14.105054802591303, 14.111103662725155, 14.03292682263308]
[4.878469814304964, 8.785605927935105, 9.024068379213155, 11.420462194837807, 13.743268393342664, 14.67094392977128, 15.465358716295906, 14.22624427797666, 17.577967363400894, 18.561235365029315, 20.854398016454727, 

In [23]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
cfg.MODEL.WEIGHTS = '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_final.pth'
predictor = DefaultPredictor(cfg)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("pedestrain_test", )
predictor = DefaultPredictor(cfg)
# COCOEvaluator.reset
evaluator = COCOEvaluator("pedestrain_test", cfg, False, output_dir = cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "pedestrain_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

4024it [00:08, 490.42it/s]


[06/27 16:11:17 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 16:11:17 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 16:11:17 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 16:11:18 d2.evaluation.evaluator]: Inference done 11/4024. 0.0397 s / img. ETA=0:02:44
[06/27 16:11:23 d2.evaluation.evaluator]: Inference done 134/4024. 0.0393 s / img. ETA=0:02:38
[06/27 16:11:28 d2.evaluation.evaluator]: Inference done 257/4024. 0.0393 s / img. ETA=0:02:33
[06/27 16:11:33 d2.evaluation.evaluator]: Inference done 380/4024. 0.0393 s / img. ETA=0:02:28
[06/27 16:11:38 d2.evaluation.evaluator]: Inference done 503/4024. 0.0394 s / img. ETA=0:02:23
[06/27 16:11:43 d2.evaluation.evaluator]: Inference done 626/4024. 0.0394 s / img. ETA=0:02:18
[06/27 16:11:48 d2.evaluation.evaluator]: Inference done 748/4024. 0.0394 s / img. ETA=0:02:13
[06/27 16:11:53 d2.evaluation.evaluator]: Inference done 869/4024. 0.0395 s / img. ETA=0:0

OrderedDict([('bbox',
              {'AP': 18.9339833705924,
               'AP50': 38.68963415587119,
               'AP75': 16.105737683223836,
               'APs': 14.03292682263308,
               'APm': 24.11692733386727,
               'APl': 31.04409102386782})])

In [10]:
# thershold 0.05
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
cfg.MODEL.WEIGHTS = '/root/notebooks/Module_final/detectron2_output/baseline_FPN_gandata_final_v2_transfer_part2/model_final.pth'
predictor = DefaultPredictor(cfg)
COCOEvaluator.reset
evaluator.reset
evaluator = COCOEvaluator("pedestrain_test", cfg, False, output_dir = cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "pedestrain_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

4024it [00:06, 665.05it/s]


[06/26 17:05:03 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/26 17:05:03 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/26 17:05:03 d2.evaluation.evaluator]: Start inference on 4024 images
[06/26 17:05:04 d2.evaluation.evaluator]: Inference done 11/4024. 0.0392 s / img. ETA=0:02:42
[06/26 17:05:09 d2.evaluation.evaluator]: Inference done 134/4024. 0.0394 s / img. ETA=0:02:38
[06/26 17:05:14 d2.evaluation.evaluator]: Inference done 254/4024. 0.0399 s / img. ETA=0:02:36
[06/26 17:05:19 d2.evaluation.evaluator]: Inference done 373/4024. 0.0401 s / img. ETA=0:02:31
[06/26 17:05:24 d2.evaluation.evaluator]: Inference done 493/4024. 0.0402 s / img. ETA=0:02:27
[06/26 17:05:29 d2.evaluation.evaluator]: Inference done 616/4024. 0.0400 s / img. ETA=0:02:21
[06/26 17:05:34 d2.evaluation.evaluator]: Inference done 737/4024. 0.0400 s / img. ETA=0:02:16
[06/26 17:05:39 d2.evaluation.evaluator]: Inference done 860/4024. 0.0399 s / img. ETA=0:0

OrderedDict([('bbox',
              {'AP': 18.38144076495051,
               'AP50': 43.76134805489884,
               'AP75': 11.392025102957385,
               'APs': 14.013466726945028,
               'APm': 23.1193958993221,
               'APl': 26.745932399031336})])

In [18]:
[OrderedDict([('bbox', {'AP': 5.977401029022199, 'AP50': 16.515525199026715, 'AP75': 2.7137084402071054, 'APs': 4.056420168893206, 'APm': 8.6148955134905, 'APl': 6.408698620682529})]), OrderedDict([('bbox', {'AP': 8.256512743095017, 'AP50': 22.301810440182997, 'AP75': 4.146245904587822, 'APs': 5.2788983272809755, 'APm': 12.002803793609457, 'APl': 9.41316462085105})]), OrderedDict([('bbox', {'AP': 9.533843952068036, 'AP50': 25.650588378481043, 'AP75': 4.6052019844605825, 'APs': 6.437489677227972, 'APm': 13.093933190206942, 'APl': 14.266285752377536})]), OrderedDict([('bbox', {'AP': 12.04775131080953, 'AP50': 30.39459104068145, 'AP75': 7.0200537405292875, 'APs': 8.328443395413718, 'APm': 16.252835718915055, 'APl': 18.53217537136591})]), OrderedDict([('bbox', {'AP': 12.976770501037302, 'AP50': 32.58854481513674, 'AP75': 7.186788728421925, 'APs': 8.648476924138185, 'APm': 17.40189230800296, 'APl': 21.178974559941913})]), OrderedDict([('bbox', {'AP': 14.859103217260236, 'AP50': 35.99676591160853, 'AP75': 9.0172377136904, 'APs': 11.363962422018242, 'APm': 19.46286428326196, 'APl': 22.094855668458745})]), OrderedDict([('bbox', {'AP': 15.350467884733227, 'AP50': 37.022957611781706, 'AP75': 9.194896097911293, 'APs': 11.386883659056625, 'APm': 19.8780629073474, 'APl': 24.165446955096275})]), OrderedDict([('bbox', {'AP': 16.132217832812277, 'AP50': 39.113607457580216, 'AP75': 9.919095092289384, 'APs': 12.059295461257925, 'APm': 20.677355278072692, 'APl': 21.93156765733089})]), OrderedDict([('bbox', {'AP': 18.067240726543062, 'AP50': 42.724219783011556, 'AP75': 11.498424942492806, 'APs': 13.761773921468956, 'APm': 22.076048585096537, 'APl': 28.642366391581742})]), OrderedDict([('bbox', {'AP': 18.38144076495051, 'AP50': 43.76134805489884, 'AP75': 11.392025102957385, 'APs': 14.013466726945028, 'APm': 23.1193958993221, 'APl': 26.745932399031336})]), OrderedDict([('bbox', {'AP': 18.38144076495051, 'AP50': 43.76134805489884, 'AP75': 11.392025102957385, 'APs': 14.013466726945028, 'APm': 23.1193958993221, 'APl': 26.745932399031336})])]

NameError: name 'OrderedDict' is not defined

In [33]:
[([('bbox', {'AP': 5.977401029022199, 'AP50': 16.515525199026715, 'AP75': 2.7137084402071054, 'APs': 4.056420168893206, 'APm': 8.6148955134905, 'APl': 6.408698620682529})]), ([('bbox', {'AP': 8.256512743095017, 'AP50': 22.301810440182997, 'AP75': 4.146245904587822, 'APs': 5.2788983272809755, 'APm': 12.002803793609457, 'APl': 9.41316462085105})]), ([('bbox', {'AP': 9.533843952068036, 'AP50': 25.650588378481043, 'AP75': 4.6052019844605825, 'APs': 6.437489677227972, 'APm': 13.093933190206942, 'APl': 14.266285752377536})]), ([('bbox', {'AP': 12.04775131080953, 'AP50': 30.39459104068145, 'AP75': 7.0200537405292875, 'APs': 8.328443395413718, 'APm': 16.252835718915055, 'APl': 18.53217537136591})]), ([('bbox', {'AP': 12.976770501037302, 'AP50': 32.58854481513674, 'AP75': 7.186788728421925, 'APs': 8.648476924138185, 'APm': 17.40189230800296, 'APl': 21.178974559941913})]), ([('bbox', {'AP': 14.859103217260236, 'AP50': 35.99676591160853, 'AP75': 9.0172377136904, 'APs': 11.363962422018242, 'APm': 19.46286428326196, 'APl': 22.094855668458745})]), ([('bbox', {'AP': 15.350467884733227, 'AP50': 37.022957611781706, 'AP75': 9.194896097911293, 'APs': 11.386883659056625, 'APm': 19.8780629073474, 'APl': 24.165446955096275})]), ([('bbox', {'AP': 16.132217832812277, 'AP50': 39.113607457580216, 'AP75': 9.919095092289384, 'APs': 12.059295461257925, 'APm': 20.677355278072692, 'APl': 21.93156765733089})]), ([('bbox', {'AP': 18.067240726543062, 'AP50': 42.724219783011556, 'AP75': 11.498424942492806, 'APs': 13.761773921468956, 'APm': 22.076048585096537, 'APl': 28.642366391581742})]), ([('bbox', {'AP': 18.38144076495051, 'AP50': 43.76134805489884, 'AP75': 11.392025102957385, 'APs': 14.013466726945028, 'APm': 23.1193958993221, 'APl': 26.745932399031336})]), ([('bbox', {'AP': 18.38144076495051, 'AP50': 43.76134805489884, 'AP75': 11.392025102957385, 'APs': 14.013466726945028, 'APm': 23.1193958993221, 'APl': 26.745932399031336})])]

[[('bbox',
   {'AP': 5.977401029022199,
    'AP50': 16.515525199026715,
    'AP75': 2.7137084402071054,
    'APs': 4.056420168893206,
    'APm': 8.6148955134905,
    'APl': 6.408698620682529})],
 [('bbox',
   {'AP': 8.256512743095017,
    'AP50': 22.301810440182997,
    'AP75': 4.146245904587822,
    'APs': 5.2788983272809755,
    'APm': 12.002803793609457,
    'APl': 9.41316462085105})],
 [('bbox',
   {'AP': 9.533843952068036,
    'AP50': 25.650588378481043,
    'AP75': 4.6052019844605825,
    'APs': 6.437489677227972,
    'APm': 13.093933190206942,
    'APl': 14.266285752377536})],
 [('bbox',
   {'AP': 12.04775131080953,
    'AP50': 30.39459104068145,
    'AP75': 7.0200537405292875,
    'APs': 8.328443395413718,
    'APm': 16.252835718915055,
    'APl': 18.53217537136591})],
 [('bbox',
   {'AP': 12.976770501037302,
    'AP50': 32.58854481513674,
    'AP75': 7.186788728421925,
    'APs': 8.648476924138185,
    'APm': 17.40189230800296,
    'APl': 21.178974559941913})],
 [('bbox',
   {

In [ ]:
[3.0228896654525625, 5.324153672946869, 5.883921125721576, 7.498841146150956, 9.511271623711961, 10.098502961924897, 10.651938517694015, 10.512437147892891, 13.551946929841902, 13.560070159877636]
[35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000]